In [26]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn import linear_model as lm
import pandas as pd
import matplotlib.pyplot as plt

In [23]:
dataset = pd.read_csv("crime.csv")

,state,murder_rate,poverty,high_school,college,single_parent,unemployed,metropolitan,region,North Central,Northeast,South,West
0,Alabama,7.4,14.7,77.5,20.4,26.0,4.6,70.2,South,0,0,1,0
1,Alaska,4.3,8.4,90.4,28.1,23.2,6.6,41.6,West,0,0,0,1
2,Arizona,7.0,13.5,85.1,24.6,23.5,3.9,87.9,West,0,0,0,1
3,Arkansas,6.3,15.8,81.7,18.4,24.7,4.4,49.0,South,0,0,1,0
4,California,6.1,14.0,81.2,27.5,21.8,4.9,96.7,West,0,0,0,1


In [31]:
### classical statistics approach

# forward selection
null_lm = ols('murder_rate ~ 1', data=dataset).fit()

single_var_models = []
single_var_models.append(ols('murder_rate ~ poverty', data=dataset).fit())
single_var_models.append(ols('murder_rate ~ high_school', data=dataset).fit())
single_var_models.append(ols('murder_rate ~ college', data=dataset).fit())
single_var_models.append(ols('murder_rate ~ single_parent', data=dataset).fit())
single_var_models.append(ols('murder_rate ~ unemployed', data=dataset).fit())
single_var_models.append(ols('murder_rate ~ metropolitan', data=dataset).fit())
single_var_models.append(ols('murder_rate ~ region', data=dataset).fit()) # baseline: north-central

for model in single_var_models:
    print(sm.stats.anova_lm(model, typ=3))

               sum_sq    df          F    PR(>F)
Intercept    0.565516   1.0   0.107489  0.744447
poverty     56.224089   1.0  10.686658  0.001999
Residual   252.535111  48.0        NaN       NaN
                 sum_sq    df          F        PR(>F)
Intercept    145.505447   1.0  35.493964  2.909426e-07
high_school  111.985984   1.0  27.317372  3.710570e-06
Residual     196.773216  48.0        NaN           NaN
               sum_sq    df          F    PR(>F)
Intercept   92.095520   1.0  15.168256  0.000304
college     17.322599   1.0   2.853055  0.097687
Residual   291.436601  48.0        NaN       NaN
                   sum_sq    df          F        PR(>F)
Intercept       57.354449   1.0  16.474017  1.808947e-04
single_parent  141.646736   1.0  40.685435  6.605019e-08
Residual       167.112464  48.0        NaN           NaN
                sum_sq    df         F    PR(>F)
Intercept     2.575526   1.0  0.456709  0.502409
unemployed   38.072110   1.0  6.751195  0.012405
Residual    2